In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 50)

target_folder_path = '/Volumes/Database/Original_data'
%cd "{target_folder_path}"

/Volumes/Database/Original_data


In [3]:
price_data = pd.read_table(rf"{target_folder_path}/台指Close.txt", encoding='Big5-HKSCS', low_memory=False)
price_data 

,期貨名稱,簡稱,日期,收盤價
0,ZTXA,台指近月期貨指數,19980721,8045.0
1,ZTXA,台指近月期貨指數,19980722,7870.0
2,ZTXA,台指近月期貨指數,19980723,7825.0
3,ZTXA,台指近月期貨指數,19980724,7835.0
4,ZTXA,台指近月期貨指數,19980727,7786.0
...,...,...,...,...
6375,ZTXA,台指近月期貨指數,20240311,19715.0
6376,ZTXA,台指近月期貨指數,20240312,19955.0
6377,ZTXA,台指近月期貨指數,20240313,19927.0
6378,ZTXA,台指近月期貨指數,20240314,19939.0


### 調整後開盤價

In [3]:
price_data = pd.read_table(rf"{target_folder_path}/股價資料庫/未調整股價.txt", encoding='Big5-HKSCS', low_memory=False)
price_data 

,證券代碼,簡稱,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),報酬率％
0,1101,台泥,19800104,24.9000,25.5,24.9,25.4,259,2.0080
1,1101,台泥,19800105,25.6000,25.8,25.4,25.7,270,1.1811
2,1101,台泥,19800107,25.7000,26.4,25.7,26.0,312,1.1673
3,1101,台泥,19800108,26.0000,26.3,25.8,25.8,224,-0.7692
4,1101,台泥,19800109,25.9000,26.1,25.7,26.0,159,0.7752
...,...,...,...,...,...,...,...,...,...
9338340,8934,衡平,20201111,34.0000,34.0,34.0,34.0,7,3.0303
9338341,8934,衡平,20201112,34.5000,34.5,34.1,34.1,62,0.2941
9338342,8934,衡平,20201113,34.5000,36.5,34.3,36.5,94,7.0381
9338343,8934,衡平,20201116,36.0000,36.0,36.0,36.0,30,-1.3699


In [ ]:
price_data['年月日'] = pd.to_datetime(price_data['年月日'].astype(str).str.strip(), format='%Y%m%d')
# data_full['漲跌停'] =  data_full['漲跌停'].astype(str).str.strip()
# data_full['證券代碼'] = data_full['證券代碼'].str.extract(r'^(\d+)')
string_list = ['證券代碼']
for column_name in string_list:
    price_data[column_name] = price_data[column_name].str.strip()

numeric_list = ['收盤價(元)', '成交量(千股)']
for column_name in numeric_list:
    price_data[column_name] = price_data[column_name].astype(str).str.strip()
    price_data[column_name] = pd.to_numeric(price_data[column_name], errors='coerce')


price_data.rename(columns ={'證券代碼':'symbol', '年月日':'date', '收盤價(元)':'close', 
                           '成交量(千股)':'volume'}, inplace=True)
price_data

In [3]:
data_full['年月日'] = pd.to_datetime(data_full['年月日'].astype(str).str.strip(), format='%Y%m%d')
string_list = ['證券代碼']
for column_name in string_list:
    data_full[column_name] = data_full[column_name].str.strip()

numeric_list = ['開盤價(元)']
for column_name in numeric_list:
    data_full[column_name] = data_full[column_name].astype(str).str.strip()
    data_full[column_name] = pd.to_numeric(data_full[column_name], errors='coerce')

data_full.rename(columns ={'證券代碼':'symbol', '年月日':'date',
                           '開盤價(元)':'adj_open'}, inplace=True)
data_full

,symbol,簡稱,date,adj_open
0,1101,台泥,1980-01-04,0.4446
1,1101,台泥,1980-01-05,0.4571
2,1101,台泥,1980-01-07,0.4588
3,1101,台泥,1980-01-08,0.4642
4,1101,台泥,1980-01-09,0.4624
...,...,...,...,...
9251987,8934,衡平,2020-11-11,34.0000
9251988,8934,衡平,2020-11-12,34.5000
9251989,8934,衡平,2020-11-13,34.5000
9251990,8934,衡平,2020-11-16,36.0000


In [4]:
output = 'adj_open'
unstack_adjopen = data_full.reset_index().set_index(['date','symbol'])[output].unstack()
pickle_file = os.path.join(f'{target_folder_path}', f'{output}.pkl')
with open(pickle_file, 'wb') as file:
    pickle.dump(unstack_adjopen, file)

### OHLCV_RELATIVE

In [5]:
data_full = pd.read_table(rf"{target_folder_path}/CandV.txt", encoding='Big5-HKSCS', low_memory=False)
data_full

,證券代碼,簡稱,年月日,收盤價(元),成交量(千股)
0,1101,台泥,19800104,25.4,259
1,1101,台泥,19800105,25.7,270
2,1101,台泥,19800107,26.0,312
3,1101,台泥,19800108,25.8,224
4,1101,台泥,19800109,26.0,159
...,...,...,...,...,...
9251987,8934,衡平,20201111,34.0,7
9251988,8934,衡平,20201112,34.1,62
9251989,8934,衡平,20201113,36.5,94
9251990,8934,衡平,20201116,36.0,30


In [6]:
data_full['年月日'] = pd.to_datetime(data_full['年月日'].astype(str).str.strip(), format='%Y%m%d')
# data_full['漲跌停'] =  data_full['漲跌停'].astype(str).str.strip()
# data_full['證券代碼'] = data_full['證券代碼'].str.extract(r'^(\d+)')
string_list = ['證券代碼']
for column_name in string_list:
    data_full[column_name] = data_full[column_name].str.strip()

numeric_list = ['收盤價(元)', '成交量(千股)']
for column_name in numeric_list:
    data_full[column_name] = data_full[column_name].astype(str).str.strip()
    data_full[column_name] = pd.to_numeric(data_full[column_name], errors='coerce')


data_full.rename(columns ={'證券代碼':'symbol', '年月日':'date', '收盤價(元)':'close', 
                           '成交量(千股)':'volume'}, inplace=True)
data_full

,symbol,簡稱,date,close,volume
0,1101,台泥,1980-01-04,25.4,259
1,1101,台泥,1980-01-05,25.7,270
2,1101,台泥,1980-01-07,26.0,312
3,1101,台泥,1980-01-08,25.8,224
4,1101,台泥,1980-01-09,26.0,159
...,...,...,...,...,...
9251987,8934,衡平,2020-11-11,34.0,7
9251988,8934,衡平,2020-11-12,34.1,62
9251989,8934,衡平,2020-11-13,36.5,94
9251990,8934,衡平,2020-11-16,36.0,30


In [7]:
output_list = ['close', 'volume']
for column_name in output_list:
    unstackdata_imf = data_full.reset_index().set_index(['date','symbol'])[column_name].unstack()

    pickle_file = os.path.join(f'{target_folder_path}', f'{column_name}.pkl')
    with open(pickle_file, 'wb') as file:
        pickle.dump(unstackdata_imf, file)

In [24]:
unstack_limit = data_full.reset_index().set_index(['date','symbol'])['limit'].unstack()
limit_df = unstack_limit.astype(str)
pickle_file = os.path.join(f'{target_folder_path}', 'limit.pkl')
with open(pickle_file, 'wb') as file:
    pickle.dump(limit_df, file)

### 台指資料

In [16]:
data_full = pd.read_table(rf"{target_folder_path}/台指資料_1201.txt", encoding='Big5-HKSCS', low_memory=False)
data_full

,證券代碼,年月日,開盤價,最高價,最低價,收盤價,成交張數(量),報酬率,基差,未平倉合約數,調整日參考價(元)
0,ZTXA 台指近月期貨指數,19980721,8131,8131,8036,8045,208,NaN,-95.80,137,NaN
1,ZTXA 台指近月期貨指數,19980722,7950,7950,7820,7870,360,-2.1753,11.06,279,NaN
2,ZTXA 台指近月期貨指數,19980723,7830,7859,7802,7825,190,-0.5718,54.89,357,NaN
3,ZTXA 台指近月期貨指數,19980724,7750,7835,7740,7835,340,0.1278,67.40,420,NaN
4,ZTXA 台指近月期貨指數,19980727,7830,7835,7780,7786,150,-0.6254,104.30,466,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6308,ZTXA 台指近月期貨指數,20231124,17330,17350,17265,17293,51647,-0.0636,-5.58,87084,NaN
6309,ZTXA 台指近月期貨指數,20231127,17257,17361,17145,17173,90354,-0.6997,-34.58,86345,NaN
6310,ZTXA 台指近月期貨指數,20231128,17215,17402,17195,17364,87471,1.0948,-18.75,88448,NaN
6311,ZTXA 台指近月期貨指數,20231129,17339,17450,17314,17334,79086,-0.1210,31.56,87554,NaN


In [17]:
data_full['年月日'] = pd.to_datetime(data_full['年月日'].astype(str).str.strip(), format='%Y%m%d')
string_list = ['證券代碼']
for column_name in string_list:
    data_full[column_name] = data_full[column_name].str.strip()

numeric_list = ['收盤價']
for column_name in numeric_list:
    data_full[column_name] = data_full[column_name].astype(str).str.strip()
    data_full[column_name] = pd.to_numeric(data_full[column_name], errors='coerce')

data_full.rename(columns ={'證券代碼':'symbol', '年月日':'date', '收盤價':'ZTXA_close'}, inplace=True)
data_full

,symbol,date,開盤價,最高價,最低價,ZTXA_close,成交張數(量),報酬率,基差,未平倉合約數,調整日參考價(元)
0,ZTXA 台指近月期貨指數,1998-07-21,8131,8131,8036,8045,208,NaN,-95.80,137,NaN
1,ZTXA 台指近月期貨指數,1998-07-22,7950,7950,7820,7870,360,-2.1753,11.06,279,NaN
2,ZTXA 台指近月期貨指數,1998-07-23,7830,7859,7802,7825,190,-0.5718,54.89,357,NaN
3,ZTXA 台指近月期貨指數,1998-07-24,7750,7835,7740,7835,340,0.1278,67.40,420,NaN
4,ZTXA 台指近月期貨指數,1998-07-27,7830,7835,7780,7786,150,-0.6254,104.30,466,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6308,ZTXA 台指近月期貨指數,2023-11-24,17330,17350,17265,17293,51647,-0.0636,-5.58,87084,NaN
6309,ZTXA 台指近月期貨指數,2023-11-27,17257,17361,17145,17173,90354,-0.6997,-34.58,86345,NaN
6310,ZTXA 台指近月期貨指數,2023-11-28,17215,17402,17195,17364,87471,1.0948,-18.75,88448,NaN
6311,ZTXA 台指近月期貨指數,2023-11-29,17339,17450,17314,17334,79086,-0.1210,31.56,87554,NaN


In [13]:
output = 'ZTXA_close'
unstack_adjopen = data_full.reset_index().set_index(['date','symbol'])[output].unstack()
pickle_file = os.path.join(f'{target_folder_path}', f'{output}.pkl')
with open(pickle_file, 'wb') as file:
    pickle.dump(unstack_adjopen, file)

KeyError: 'ZTXA_close'

In [14]:
data_full = pd.read_table(rf"{target_folder_path}/limit.txt", encoding='Big5-HKSCS', low_memory=False)
data_full

,證券代碼,簡稱,年月日,漲跌停
0,1101,台泥,19800104,
1,1101,台泥,19800105,
2,1101,台泥,19800107,
3,1101,台泥,19800108,
4,1101,台泥,19800109,
...,...,...,...,...
9251987,8934,衡平,20201111,
9251988,8934,衡平,20201112,
9251989,8934,衡平,20201113,
9251990,8934,衡平,20201116,


In [15]:
data_full['年月日'] = pd.to_datetime(data_full['年月日'].astype(str).str.strip(), format='%Y%m%d')
data_full['漲跌停'] =  data_full['漲跌停'].astype(str).str.strip()
string_list = ['證券代碼']
for column_name in string_list:
    data_full[column_name] = data_full[column_name].str.strip()


data_full.rename(columns ={'證券代碼':'symbol', '年月日':'date', '漲跌停':'limit', 
                           }, inplace=True)
data_full

,symbol,簡稱,date,limit
0,1101,台泥,1980-01-04,
1,1101,台泥,1980-01-05,
2,1101,台泥,1980-01-07,
3,1101,台泥,1980-01-08,
4,1101,台泥,1980-01-09,
...,...,...,...,...
9251987,8934,衡平,2020-11-11,
9251988,8934,衡平,2020-11-12,
9251989,8934,衡平,2020-11-13,
9251990,8934,衡平,2020-11-16,


In [16]:
# 漲跌停資料另外處理

unstack_limit = data_full.reset_index().set_index(['date','symbol'])['limit'].unstack()
unstack_limit

def convert_to_boolean(x):
    if x == '+' or x == '-':
        return False
    elif pd.isna(x):  # 檢查NaN
        return True
    return True  # 如果有其他未知值，保持原樣
limit_df = unstack_limit.applymap(convert_to_boolean)
limit_df

pickle_file = os.path.join(f'{target_folder_path}', 'limit.pkl')
with open(pickle_file, 'wb') as file:
    pickle.dump(limit_df, file)

/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_14748/2739152200.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  limit_df = unstack_limit.applymap(convert_to_boolean)
